
# JD Equipment Classifier (Keras, Synthetic Dataset)

This notebook trains a **5-class image classifier** (tractor, combine, sprayer, skidsteer, baler) using the synthetic data you generated:
- Images: `/mnt/data/jd_images.npy` (N, 128, 128, 3)
- Labels CSV: `/mnt/data/jd_labels.csv` with columns: `image_id, class_name, class_id, split`

> You can later swap to real images with the same pipeline.


In [ ]:

import os, json, math, random, numpy as np, pandas as pd
import matplotlib.pyplot as plt

DATA_IMAGES = "/mnt/data/jd_images.npy"
DATA_LABELS = "/mnt/data/jd_labels.csv"

assert os.path.exists(DATA_IMAGES), f"Missing: {DATA_IMAGES}"
assert os.path.exists(DATA_LABELS), f"Missing: {DATA_LABELS}"

print("Found:", DATA_IMAGES, DATA_LABELS)


In [ ]:

# Hyperparameters
IMG_SIZE = (128, 128)        # matches the synthetic generator
NUM_CLASSES = 5              # tractor, combine, sprayer, skidsteer, baler
BATCH_SIZE = 32
EPOCHS = 3                   # increase when running on GPU
LR = 1e-3
MODEL_OUT = "/mnt/data/jd_equipment_classifier.h5"
CLASS_INDEX_JSON = "/mnt/data/jd_class_index.json"


In [ ]:

# Load arrays and labels
X = np.load(DATA_IMAGES)                 # uint8
y_df = pd.read_csv(DATA_LABELS)         # includes class_id and split

# Basic checks
print(X.shape, X.dtype)
print(y_df.head())

# Normalize images to [0,1]
X = X.astype("float32") / 255.0
y = y_df["class_id"].astype("int32").values

# Build split masks
train_mask = (y_df["split"]=="train").values
val_mask   = (y_df["split"]=="val").values
test_mask  = (y_df["split"]=="test").values

X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

print("Splits:", X_train.shape, X_val.shape, X_test.shape)
cls_names = sorted(y_df.drop_duplicates("class_id")[["class_id","class_name"]].values.tolist(), key=lambda x: x[0])
id2name = {int(i): n for i,n in cls_names}
name2id = {n:i for i,n in id2name.items()}
id2name


In [ ]:

# Build tf.data pipelines
import tensorflow as tf

def make_ds(X, y, batch, training=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(min(10000, len(X)))
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(X_train, y_train, BATCH_SIZE, training=True)
val_ds   = make_ds(X_val,   y_val,   BATCH_SIZE, training=False)
test_ds  = make_ds(X_test,  y_test,  BATCH_SIZE, training=False)

len_train = len(list(iter(train_ds)))
print("Batches per epoch (train):", len_train)


In [ ]:

# Model: EfficientNetB0 for speed (can bump to B3 if you have GPU)
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

base = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), weights=None)
# Note: weights=None because this is a tiny synthetic dataset. You can try ImageNet weights for real photos.
base.trainable = True

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


In [ ]:

# Train
hist = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)


In [ ]:

# Evaluate
test_loss, test_acc = model.evaluate(test_ds)
print("Test acc:", test_acc)


In [ ]:

# Save model and class index mapping
model.save(MODEL_OUT)
with open(CLASS_INDEX_JSON, "w") as f:
    json.dump(id2name, f, indent=2)
print("Saved:", MODEL_OUT, CLASS_INDEX_JSON)



### Quick inference utility
Upload or pick an image from the synthetic arrays and run a prediction.


In [ ]:

def predict_one(img):
    # img should be float32 [0,1], shape (H,W,3)
    import numpy as np
    arr = np.expand_dims(img, 0)
    probs = model.predict(arr, verbose=0)[0]
    top = int(np.argmax(probs))
    return top, float(probs[top]), probs

# Try a few from test split
import matplotlib.pyplot as plt
idxs = np.where(test_mask)[0][:8]
plt.figure(figsize=(12,6))
for i,k in enumerate(idxs[:8]):
    plt.subplot(2,4,i+1)
    pred, p, probs = predict_one(X[k])
    plt.imshow(X[k])
    plt.axis("off")
    plt.title(f"pred={id2name[pred]} ({p:.2f})\ntrue={id2name[int(y[k])]}")
plt.tight_layout()
plt.show()



### Grad-CAM (optional explainability)
This visualizes *where* the model is focusing. (Crude but useful for demos.)


In [ ]:

# Lightweight Grad-CAM
import numpy as np, tensorflow as tf, matplotlib.pyplot as plt

# pick the last conv layer from EfficientNetB0
last_conv = None
for layer in model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv = layer.name
        break
if last_conv is None:
    # fallback: search inside the EfficientNet base
    for layer in base.layers[::-1]:
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv = layer.name
            break
print("Using last conv layer:", last_conv)

def grad_cam(img, class_index=None):
    img_in = tf.expand_dims(img, 0)
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_in)
        if class_index is None:
            class_index = tf.argmax(preds[0])
        class_channel = preds[:, class_index]
    grads = tape.gradient(class_channel, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_out = conv_out[0]
    heatmap = conv_out @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def overlay_heatmap(img, heatmap, alpha=0.4):
    import cv2
    hm = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    hm = np.uint8(255 * hm)
    hm = cv2.applyColorMap(hm, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(cv2.cvtColor((img*255).astype(np.uint8), cv2.COLOR_RGB2BGR), 1.0, hm, alpha, 0)
    return cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

# Demo on one test image
k = int(np.where(test_mask)[0][0])
pred, p, _ = predict_one(X[k])
hm = grad_cam(X[k], class_index=pred)
ov = overlay_heatmap(X[k], hm, alpha=0.45)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(X[k]); plt.axis("off"); plt.title(f"Input\ntrue={id2name[int(y[k])]}, pred={id2name[pred]}")
plt.subplot(1,2,2); plt.imshow(ov); plt.axis("off"); plt.title("Grad-CAM overlay")
plt.tight_layout(); plt.show()
